IMPORTING LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz as sv

from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector as selector
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn import linear_model

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
import smogn

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import ModelCheckpoint

SEED=42

FUNCTION FOR EVALUATING MODEL PERFORMANCE 

In [ ]:
def evaluate_model(model, y_test, y_pred, y_train, X_train, X_test):
    print('MAE: ', mean_absolute_error(y_test, y_pred))
    print('MSE: ', mean_squared_error(y_test, y_pred)) 
    print('Training set score: {:.4f}'.format(model.score(X_train, y_train)))
    print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

FUNCTION FOR PLOTTING y_pred VS y_test 

In [ ]:
def plot_results(y_test, y_pred):
    results = pd.DataFrame(zip(y_test, y_pred, y_test - y_pred), columns = ['y_test', 'y_pred', 'error'])

    plt.figure(figsize=(10,10))
    x=np.linspace(0,45,45)
    plt.plot(results['y_test'], results['y_pred'], 'b.')
    plt.plot(x, x, 'r-')
    plt.xlim(0,45)
    plt.ylim(0,45)
    plt.title("Results", fontsize=16)
    plt.xlabel("Real", fontsize=14)
    plt.ylabel("Predicted", fontsize=14)
    plt.show()

CHECKING FEATURE IMPORTANCES

In [ ]:
# read in data
df = pd.read_excel('majus29_ALL_SEER.xlsx', sheet_name='ord_cols', header=0)

# converting HIST_TYPE because it has been coded as integer
convert_dict = {'Histologic Type ICD-O-3': object}
df = df.astype(convert_dict)

In [ ]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(df)

data_categorical = df[categorical_columns]
df_non_cat = df.drop(categorical_columns, axis=1)

encoder = OrdinalEncoder()
df_encoded = pd.DataFrame(encoder.fit_transform(data_categorical.astype(str)))
df_encoded.columns = categorical_columns
df_enc = pd.concat([df_encoded, df_non_cat], axis=1)

In [ ]:
y = df_enc['Survival months']
X = df_enc.drop(['Survival months'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=SEED)

rfr = RandomForestRegressor()

# Fitting model to train data
rfr.fit(X_train, y_train)

importances_df = pd.DataFrame({"Feature_names" : rfr.feature_names_in_, 
                               "Importances" : rfr.feature_importances_})
importances_df = importances_df[importances_df['Importances'] >= 0.01]

g = sns.barplot(data=importances_df, 
                x="Importances", 
                y="Feature_names",
                palette='mako',
                order=importances_df.sort_values('Importances', ascending=False).Feature_names)
sns.despine(bottom=True, left=True)

# setting x ticks as empty
g.set(xticks=[])
g.set_title("Feature importances", fontsize=14)
for value in g.containers:
    g.bar_label(value, padding=2)# Obtaining feature importances

g.figure.set_size_inches(6,6)

PRE-PROCESSING

In [ ]:
df = pd.read_excel('majus29_ALL_SEER.xlsx', sheet_name='ord_cols', header=0)

In [ ]:
df['TUMOR_SIZE_mm'] = df['TUMOR_SIZE_cm_earlier'].fillna(df['TUMOR_SIZE_cm_later'])
df = df.drop(['TUMOR_SIZE_cm_earlier', 'TUMOR_SIZE_cm_later'], axis=1)

inc_dict = {'$75,000+':0.1, '$70,000 - $74,999': 0.2, '$65,000 - $69,999': 0.3, '$60,000 - $64,999':0.4, '$55,000 - $59,999':0.5, '$45,000 - $49,999':0.6, '$40,000 - $44,999':0.7, '$50,000 - $54,999':0.8, '$35,000 - $39,999':0.9, '< $35,000':1, 'Unknown/missing/no match/Not 1990-2018': np.NaN}
df['MEDIAN_INCOME_dollars'] = df['MEDIAN_INCOME_dollars'].map(inc_dict)

gr_dict = {'Moderately differentiated; Grade II':0.3, 'Unknown': np.NaN, 'Poorly differentiated; Grade III': 0.6, 'Well differentiated; Grade I': 0, 'Undifferentiated; anaplastic; Grade IV':1, 'Blank(s)':np.NaN}
df['GRADE'] = df['GRADE'].map(gr_dict)

st_dict = {'Regional': 0.5,'Localized':0, 'Unknown/unstaged': np.NaN, 'Distant': 1, 'Blank(s)':np.NaN}
df['STAGE'] = df['STAGE'].map(st_dict)

df[['HISTOLOGIC_TYPE_ICD_O_3', 'PRIMARY_SITE']] = df[['HISTOLOGIC_TYPE_ICD_O_3', 'PRIMARY_SITE']].apply(LabelEncoder().fit_transform)

cols = df.columns

In [ ]:
y = df['SURVIVAL_TIME_months']
X = df.drop(['SURVIVAL_TIME_months'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=SEED)

HGBR_model = HistGradientBoostingRegressor(random_state = SEED)
HGBR_model.fit(X_train, y_train)
y_pred = HGBR_model.predict(X_test)
evaluate_model(HGBR_model, y_test, y_pred, y_train, X_train, X_test)
plot_results(y_test, y_pred)

HANDLING MISSING VALUES

In [ ]:
imp_mean = IterativeImputer(estimator=linear_model.BayesianRidge(),n_nearest_features=None, imputation_order='ascending', random_state=SEED)
imp_mean.fit(df)
df = pd.DataFrame(imp_mean.transform(df))
df.columns = cols

SCALING NUMERICAL COLUMNS

In [ ]:
sc_age = StandardScaler()
df['AGE_AT_DIAGNOSIS_years'] = sc_age.fit_transform(df['AGE_AT_DIAGNOSIS_years'].values.reshape(-1,1))

sc_size = StandardScaler()
df['TUMOR_SIZE_mm'] = sc_size.fit_transform(df['TUMOR_SIZE_mm'].values.reshape(-1,1))

sc_hist = StandardScaler()
df['HISTOLOGIC_TYPE_ICD_O_3'] = sc_hist.fit_transform(df['HISTOLOGIC_TYPE_ICD_O_3'].values.reshape(-1,1))

sc_site = StandardScaler()
df['PRIMARY_SITE'] = sc_site.fit_transform(df['PRIMARY_SITE'].values.reshape(-1,1))

sc_mal = StandardScaler()
df['TOTAL_NUM_OF_MALIGNANT_TUMORS'] = sc_mal.fit_transform(df['TOTAL_NUM_OF_MALIGNANT_TUMORS'].values.reshape(-1,1))

In [ ]:
df

In [ ]:
sns.kdeplot(df, x="SURVIVAL_TIME_months")

In [ ]:
y = df['SURVIVAL_TIME_months']
X = df.drop(['SURVIVAL_TIME_months'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=SEED)

In [ ]:
RFR_model = RandomForestRegressor(random_state = SEED)
RFR_model.fit(X_train, y_train)
y_pred = RFR_model.predict(X_test)
evaluate_model(RFR_model, y_test, y_pred, y_train, X_train, X_test)
plot_results(y_test, y_pred)

SMOGN

In [ ]:
rg_mtrx = [
    [0,1,0],
    [2,0,0],
    [60,0.8,0],
    [100,0.95,0],
    [150,1,0]
]

df_smogn = smogn.smoter(
    
    ## main arguments
    data = df,           ## pandas dataframe
    y = 'SURVIVAL_TIME_months',          ## string ('header name')
    k = 8,                    ## positive integer (k < n)
    pert = 0.4,              ## real number (0 < R < 1)
    samp_method = 'extreme',  ## string ('balance' or 'extreme')
    drop_na_col = True,       ## boolean (True or False)
    drop_na_row = True,       ## boolean (True or False)
    replace = False,          ## boolean (True or False)

    ## phi relevance arguments
    rel_thres = 0.6,         ## real number (0 < R < 1)
    rel_method = 'manual',    ## string ('auto' or 'manual')
    # rel_xtrm_type = 'both', ## unused (rel_method = 'manual')
    # rel_coef = 1.50,        ## unused (rel_method = 'manual')
    rel_ctrl_pts_rg = rg_mtrx ## 2d array (format: [x, y])
)

RANDOM FOREST

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 3)]
# Number of features to consider at every split
max_features = [None, 0.5]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 40, num = 2)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestRegressor(random_state=SEED)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=SEED, n_jobs = -1)
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
y = df_smogn['SURVIVAL_TIME_months']
X = df_smogn.drop(['SURVIVAL_TIME_months'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=SEED)

RFR_model = RandomForestRegressor(n_estimators = 200, min_samples_split = 5, min_samples_leaf = 1, max_features = 0.5, max_depth = None, bootstrap = False, random_state = SEED)
RFR_model.fit(X_train, y_train)
y_pred = RFR_model.predict(X_test)
evaluate_model(RFR_model, y_test, y_pred, y_train, X_train, X_test)
plot_results(y_test, y_pred)

XGBOOST

In [ ]:
XGB_model = xgb.XGBRegressor(random_state=SEED)
XGB_model.fit(X_train, y_train)
y_pred = XGB_model.predict(X_test)
evaluate_model(XGB_model, y_test, y_pred, y_train, X_train, X_test)

In [ ]:
# Hyper Parameter Optimization
n_estimators = [100, 500, 900]
max_depth = [2, 3, 5]
booster=['gbtree']
learning_rate=[0.05,0.1,0.15]
min_child_weight=[1,2]
base_score=[0.5,0.75,1]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

# Set up the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator=XGB_model, param_distributions=hyperparameter_grid, cv=3, n_iter=50, scoring = 'neg_mean_absolute_error', n_jobs = 4, verbose = 5, return_train_score = True, random_state=SEED)

random_cv.fit(X_train,y_train)

In [ ]:
random_cv.best_estimator_

In [ ]:
XGB_model = xgb.XGBRegressor(base_score=0.75, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.15, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, monotone_constraints=None,
             n_estimators=900, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=SEED)
XGB_model.fit(X_train, y_train)
y_pred_XGB = XGB_model.predict(X_test)
evaluate_model(XGB_model, y_test, y_pred_XGB, y_train, X_train, X_test)
plot_results(y_test, y_pred_XGB)

NEURAL NETWORK

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split = 0.3, callbacks=callbacks_list)

In [ ]:
wights_file = 'Weights-497--4.10621.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
y_pred_NN = np.reshape(NN_model.predict(X_test), (2434,))
print('MAE: ', mean_absolute_error(y_test, y_pred_NN))
print('MSE: ', mean_squared_error(y_test, y_pred_NN))